这里说频率是指一个机场航班出现的频率

In [1]:
# 使用贝叶斯优化替代上方的手动超参数训练
from xgb_bayes_opt import bayesian_tune_xgb, smape_eval

base_params = {
    'objective': 'reg:squarederror',
}

# 上文已定义 dtrain/dval/dtest，使用验证集作为优化目标
evals = [(dtrain, 'train'), (dval, 'validation')]

model_bayes, best_params, best_smape = bayesian_tune_xgb(
    dtrain,
    evals,
    base_params=base_params,
    init_points=8,
    n_iter=25,
    early_stopping_rounds=10,
    verbose_eval=10,
    save_model_name='xgb_model_bayes.json',
)

print('Best SMAPE:', best_smape)
print('Best Params:', best_params)

# 使用最优模型进行测试集预测
y_pred_bayes = model_bayes.predict(dtest)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# 使用贝叶斯优化进行XGBoost超参数调优
# 需要安装: pip install bayesian-optimization
from xgb_bayes_opt import bayesian_tune_xgb, smape_eval

base_params = {
    'objective': 'reg:squarederror',
}

# 使用验证集作为优化目标
evals = [(dtrain, 'train'), (dval, 'validation')]

model_bayes, best_params, best_smape = bayesian_tune_xgb(
    dtrain,
    evals,
    base_params=base_params,
    init_points=8,
    n_iter=25,
    early_stopping_rounds=10,
    verbose_eval=10,
    save_model_name='xgb_model_bayes.json',
)

print('Best SMAPE:', best_smape)
print('Best Params:', best_params)

# 使用最优模型进行测试集预测
y_pred_bayes = model_bayes.predict(dtest)


## 加载数据

数据已经预处理过了

In [21]:
import pandas as pd

# 加载数据
data = pd.read_csv('data_hh/result/pre_2023-2025_with_comp_train.csv', dtype={'flt_no': str})

# 查看前几行数据，确保加载成功
# 显示前两行数据以确保正确加载
print(data.shape)
print(data.head(5))
print(data.tail(5))

(1689609, 27)
   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705  110.0      195     1       1      1.30   97  a9f275  24cc4b   
1  6a67d9  110.0      195     1       1      1.25   67  d56796  24cc4b   
2  de569b   93.0      190     3       3      4.75   22  d56796  ec1b13   
3  284078  162.0      320     1       1      1.83   55  e3d141  d618e3   
4  f8765b  167.0      320     1       1      3.98  166  9550bc  2e1ea1   

        c   unit_price  competitor_price  year  month  day  weekday  quarter  \
0  7f9c2b   470.474227        -94.900773  2023      1    1        6        1   
1  7f9c2b   454.925373       -123.872095  2023      1    1        6        1   
2  dcf90a  1177.818182        977.818182  2023      1    1        6        1   
3  7f9c2b   669.090909          0.000000  2023      1    1        6        1   
4  7f9c2b  1794.783133       1564.978621  2023      1    1        6        1   

   is_weekend  is_holiday_season  month_sin  month_cos  hour

In [22]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    1.689609e+06
mean     1.222227e+02
std      5.222077e+01
min      2.000000e+01
25%      8.300000e+01
50%      1.280000e+02
75%      1.580000e+02
max      3.110000e+02
Name: pax, dtype: float64


## 编码分类变量

### 新增城市标签

In [23]:
import json
# 加载字典
with open('data_hh/result/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'000': 1, '13f': 0, '8e3': 0, 'a18': 0, 'eda': 2, '01a': 1, 'b5a': 0, 'ea1': 0, '03f': 2, '1be': 0, '3bc': 0, '578': 0, '5df': 0, '5fb': 0, '760': 0, '792': 0, '7ff': 0, '90a': 0, '974': 0, 'bbb': 0, 'cb5': 0, 'ffe': 0, '059': 2, '022': 0, '077': 0, '0d7': 0, '0dd': 0, '1a3': 0, '1e9': 0, '1ed': 0, '25c': 0, '311': 0, '354': 0, '451': 0, '510': 0, '5d7': 0, '63a': 0, '666': 0, '6e3': 0, '7f1': 0, '812': 0, '8a1': 0, '9d4': 0, '9e9': 0, 'b3a': 0, 'bff': 0, 'c24': 0, 'c3a': 0, 'c4c': 0, 'c98': 0, 'c9f': 0, 'cb7': 0, 'd7c': 0, 'e76': 0, 'e9c': 0, 'efb': 0, '05c': 2, '141': 0, '157': 0, '709': 0, '8ac': 0, '97f': 0, '988': 0, 'a2f': 0, 'b09': 0, 'c14': 0, 'c4b': 0, 'e5b': 0, 'ec2': 0, '074': 1, '4d0': 0, '695': 0, 'c2c': 0, 'fa9': 0, '08c': 1, '08f': 2, '3c5': 0, '5e8': 0, 'a72': 0, 'd54': 0, 'ff1': 0, '0ac': 1, '0c8': 1, '0ca': 1, 'e09': 0, '0d1': 1, '13e': 0, '1ee': 0, '43e': 0, '0db': 2, '919': 0, 'd08': 0, '0ff': 2, '11b': 1, '123': 2, '5de': 0, '8eb': 0, '124': 1, '130': 2, '1

In [24]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [25]:
import json

# 加载字典
with open('data_hh/result/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'000': [-1.6952509880065918, -0.8259809613227844], '13f': [0.7161105871200562, -0.43590694665908813], '8e3': [0.5429399609565735, -0.520667314529419], 'a18': [0.20013581216335297, -0.4142817258834839], 'eda': [-0.15405425429344177, -1.1048771142959595], '01a': [-1.7130950689315796, 0.15070012211799622], 'b5a': [0.5836260318756104, -0.7250841856002808], 'ea1': [0.2437203973531723, 0.056170180439949036], '03f': [-1.0081822872161865, -0.733413577079773], '1be': [0.8263573050498962, -0.8512860536575317], '3bc': [1.4952945709228516, -1.2341231107711792], '578': [1.0146775245666504, -0.5335901975631714], '5df': [0.842978298664093, -0.9285436272621155], '5fb': [0.7287545204162598, -0.4136313796043396], '760': [1.2267078161239624, -0.6878513693809509], '792': [0.1306421011686325, -0.7094545364379883], '7ff': [0.9274259805679321, -0.45611515641212463], '90a': [0.06965449452400208, -0.2664845883846283], '974': [1.24435293674469, -1.1002029180526733], 'bbb': [0.4501871168613434, -0.740858

In [26]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])


         a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0                  NaN            NaN            NaN            NaN   
1                  NaN            NaN            NaN            NaN   
2                  NaN            NaN            NaN            NaN   
3                  NaN            NaN            NaN            NaN   
4                  NaN            NaN            NaN            NaN   
...                ...            ...            ...            ...   
1689604            NaN            NaN            NaN            NaN   
1689605            NaN            NaN            NaN            NaN   
1689606            NaN            NaN            NaN            NaN   
1689607            NaN            NaN            NaN            NaN   
1689608            NaN            NaN            NaN            NaN   

         c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                  NaN            NaN         -1.641072          1.691

### 频率编码

使用 json 保存和加载 city_map

In [27]:
# 加载city_map
with open('data_hh/result/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

         flt_no    cap aircraft  legs  leg_no  duration  pax   a   b   c  \
0        3c6705  110.0      195     1       1      1.30   97 NaN NaN NaN   
1        6a67d9  110.0      195     1       1      1.25   67 NaN NaN NaN   
2        de569b   93.0      190     3       3      4.75   22 NaN NaN NaN   
3        284078  162.0      320     1       1      1.83   55 NaN NaN NaN   
4        f8765b  167.0      320     1       1      3.98  166 NaN NaN NaN   
...         ...    ...      ...   ...     ...       ...  ...  ..  ..  ..   
1689604  2f5ac7  161.0      738     3       2      2.58  128 NaN NaN NaN   
1689605  8b52d8  161.0      738     1       1      2.60  152 NaN NaN NaN   
1689606  cac556  161.0      738     1       1      1.77  149 NaN NaN NaN   
1689607  0bd968  188.0      7MZ     1       1      2.40  144 NaN NaN NaN   
1689608  2befc2  161.0      738     1       1      2.48  142 NaN NaN NaN   

          unit_price  competitor_price  year  month  day  weekday  quarter  \
0        

### 'flt_no', 'bd_type', 'aircraft'编码

In [28]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no', 'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('data_hh/result/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1104         2
1    1872         2
2    3916         0
3     721         9
4    4356         9


## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [29]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
import numpy as np
# 存在性检查→缺则补（只在缺失时计算）
_need = ['quarter','is_weekend','is_holiday_season','hour_sin','hour_cos','month_sin','month_cos']
_missing = [c for c in _need if c not in data.columns]
if _missing:
    assert all(col in data.columns for col in ['month','weekday','hour']), '缺少 month/weekday/hour 无法补时间特征'
    data['quarter'] = data['month'].apply(lambda x: (x - 1) // 3 + 1)
    data['is_weekend'] = data['weekday'].isin([5, 6]).astype(int)
    data['is_holiday_season'] = data['month'].isin([1, 2, 7, 8, 10]).astype(int)
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday', 'hour', 'minute', 'quarter', 'is_weekend', 'is_holiday_season', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'from', 'to', 'unit_price', 'competitor_price', 'a_label', 'b_label', 'c_label', 'from_label', 'to_label', 'a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']]

# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

### 对x进行标准化

In [30]:
from sklearn.preprocessing import StandardScaler

# 对所有特征进行标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将标准化后的数据转回 DataFrame 格式
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# 查看结果
pd.set_option('display.max_columns', None)  # 显示所有列
print(X_scaled.head(1))


# 保存 scaler
joblib.dump(scaler, 'data_hh/result/encoder/standard_scaler_x.pkl')
print("x的标准化器已保存为 standard_scaler_x.pkl")

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


     flt_no       cap  aircraft     legs    leg_no  duration   a   b   c  \
0 -0.848829 -1.494606 -1.378712 -0.85182 -0.558079 -1.236975 NaN NaN NaN   

       year     month      day   weekday      hour    minute   quarter  \
0 -0.977504 -1.307816 -1.68186  1.482253 -0.001571  0.479603 -1.051653   

   is_weekend  is_holiday_season  hour_sin  hour_cos  month_sin  month_cos  \
0    1.564266           1.333616 -0.481549 -0.919969   0.368371   1.241001   

       from        to  unit_price  competitor_price  a_label  b_label  \
0 -1.350272  0.203254   -0.567115         -0.457282      NaN      NaN   

   c_label  from_label  to_label  a_embedding_1  a_embedding_2  b_embedding_1  \
0      NaN   -3.677972  -0.20304            NaN            NaN            NaN   

   b_embedding_2  c_embedding_1  c_embedding_2  from_embedding_1  \
0            NaN            NaN            NaN         -2.424275   

   from_embedding_2  to_embedding_1  to_embedding_2  
0          3.197464        -0.76605     

### 对y进行标准化

In [31]:
# 对目标列 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # 将 y 转换为 2D 数组进行标准化

# 转换回 DataFrame 格式
y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'])

# 查看标准化后的 y
print(y_scaled.head())

# 保存 y 的 scaler
joblib.dump(scaler_y, 'data_hh/result/encoder/standard_scaler_y.pkl')
print("y的标准化器已保存为 standard_scaler_y.pkl")

   pax_scaled
0   -0.483002
1   -1.057486
2   -1.919213
3   -1.287280
4    0.838312
y的标准化器已保存为 standard_scaler_y.pkl


## 训练XGBoost模型

In [32]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 自定义 SMAPE 函数
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    return np.mean(diff / denominator) * 100

# 自定义评估函数
def smape_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    smape_value = smape(y_true, y_pred)
    return 'SMAPE', smape_value

# 数据划分
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 输出数据集大小
print(f'训练集大小: {X_train.shape[0]}')
print(f'验证集大小: {X_val.shape[0]}')
print(f'测试集大小: {X_test.shape[0]}')

# 转换为 DMatrix 格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# 使用贝叶斯优化替代手动参数设置
from xgb_bayes_opt import bayesian_tune_xgb, smape_eval

base_params = {
    'objective': 'reg:squarederror',
    # 可选: 'tree_method': 'hist', 'nthread': 8, 'seed': 42
}

# 评估集（以验证集为优化目标）
evals = [(dtrain, 'train'), (dval, 'validation')]

model, best_params, best_smape = bayesian_tune_xgb(
    dtrain,
    evals,
    base_params=base_params,
    init_points=16,
    n_iter=64,
    early_stopping_rounds=10,
    verbose_eval=10,
    save_model_name='xgb_model_bayes.json',
)

print('Best SMAPE:', best_smape)
print('Best Params:', best_params)

# 预测测试集
y_pred = model.predict(dtest)

# # 测试集 SMAPE 评估
# test_smape = smape(y_test, y_pred)
# print(f'SMAPE on Test Set: {test_smape:.2f}%')

# # 测试集 MSE 评估
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error on Test Set: {mse}')

训练集大小: 1351687
验证集大小: 168961
测试集大小: 168961
|   iter    |  target   | learni... | max_depth | subsample | colsam... |   alpha   |  lambda_  | num_bo... |
-------------------------------------------------------------------------------------------------------------


/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[0]	train-rmse:0.91134	train-SMAPE:164.66092	validation-rmse:0.90935	validation-SMAPE:164.74374
[10]	train-rmse:0.50626	train-SMAPE:79.79658	validation-rmse:0.50993	validation-SMAPE:80.55767
[20]	train-rmse:0.41330	train-SMAPE:67.58068	validation-rmse:0.42213	validation-SMAPE:68.71056
[30]	train-rmse:0.38834	train-SMAPE:63.84524	validation-rmse:0.40114	validation-SMAPE:65.32657
[40]	train-rmse:0.37529	train-SMAPE:62.14840	validation-rmse:0.39178	validation-SMAPE:63.98139
[50]	train-rmse:0.36553	train-SMAPE:60.88355	validation-rmse:0.38518	validation-SMAPE:63.03502
[60]	train-rmse:0.35881	train-SMAPE:59.97059	validation-rmse:0.38098	validation-SMAPE:62.39506
[70]	train-rmse:0.35274	train-SMAPE:59.17868	validation-rmse:0.37719	validation-SMAPE:61.86174
[80]	train-rmse:0.34836	train-SMAPE:58.59185	validation-rmse:0.37437	validation-SMAPE:61.42556
[90]	train-rmse:0.34324	train-SMAPE:57.90687	validation-rmse:0.37129	validation-SMAPE:60.97617
[100]	train-rmse:0.33871	train-SMAPE:57.29362	val

[120]	train-rmse:0.48156	train-SMAPE:74.28709	validation-rmse:0.48225	validation-SMAPE:74.65337
[130]	train-rmse:0.47961	train-SMAPE:73.98556	validation-rmse:0.48029	validation-SMAPE:74.35426
[140]	train-rmse:0.47799	train-SMAPE:73.77721	validation-rmse:0.47872	validation-SMAPE:74.14899
[150]	train-rmse:0.47660	train-SMAPE:73.58362	validation-rmse:0.47735	validation-SMAPE:73.96403
[160]	train-rmse:0.47499	train-SMAPE:73.39635	validation-rmse:0.47578	validation-SMAPE:73.77286
[170]	train-rmse:0.47386	train-SMAPE:73.24156	validation-rmse:0.47470	validation-SMAPE:73.62016
[180]	train-rmse:0.47284	train-SMAPE:73.09920	validation-rmse:0.47370	validation-SMAPE:73.47168
[190]	train-rmse:0.47147	train-SMAPE:72.95921	validation-rmse:0.47235	validation-SMAPE:73.33334
[200]	train-rmse:0.47034	train-SMAPE:72.84488	validation-rmse:0.47125	validation-SMAPE:73.22055
[210]	train-rmse:0.46927	train-SMAPE:72.72963	validation-rmse:0.47021	validation-SMAPE:73.10825
[220]	train-rmse:0.46825	train-SMAPE:72.

[130]	train-rmse:0.46922	train-SMAPE:72.98277	validation-rmse:0.46988	validation-SMAPE:73.33591
[140]	train-rmse:0.46721	train-SMAPE:72.69218	validation-rmse:0.46794	validation-SMAPE:73.06383
[150]	train-rmse:0.46568	train-SMAPE:72.51833	validation-rmse:0.46643	validation-SMAPE:72.89482
[160]	train-rmse:0.46396	train-SMAPE:72.27417	validation-rmse:0.46477	validation-SMAPE:72.66305
[170]	train-rmse:0.46241	train-SMAPE:72.09261	validation-rmse:0.46326	validation-SMAPE:72.49536
[180]	train-rmse:0.46104	train-SMAPE:71.90194	validation-rmse:0.46193	validation-SMAPE:72.31632
[190]	train-rmse:0.45951	train-SMAPE:71.71598	validation-rmse:0.46046	validation-SMAPE:72.13237
[200]	train-rmse:0.45836	train-SMAPE:71.59003	validation-rmse:0.45934	validation-SMAPE:72.00459
[210]	train-rmse:0.45708	train-SMAPE:71.41805	validation-rmse:0.45811	validation-SMAPE:71.83833
[220]	train-rmse:0.45600	train-SMAPE:71.27377	validation-rmse:0.45709	validation-SMAPE:71.69917
[230]	train-rmse:0.45477	train-SMAPE:71.

...

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[10]	train-rmse:0.47305	train-SMAPE:73.94025	validation-rmse:0.47377	validation-SMAPE:74.30263
[20]	train-rmse:0.45434	train-SMAPE:71.02708	validation-rmse:0.45584	validation-SMAPE:71.48027
[30]	train-rmse:0.44477	train-SMAPE:69.84709	validation-rmse:0.44660	validation-SMAPE:70.31935
[40]	train-rmse:0.43644	train-SMAPE:68.96493	validation-rmse:0.43872	validation-SMAPE:69.44297
[50]	train-rmse:0.42816	train-SMAPE:67.92408	validation-rmse:0.43100	validation-SMAPE:68.43860
[60]	train-rmse:0.42327	train-SMAPE:67.33418	validation-rmse:0.42647	validation-SMAPE:67.83579
[70]	train-rmse:0.41951	train-SMAPE:66.88354	validation-rmse:0.42307	validation-SMAPE:67.39770
[80]	train-rmse:0.41581	train-SMAPE:66.39861	validation-rmse:0.41986	validation-SMAPE:66.94111
[90]	train-rmse:0.41181	train-SMAPE:65.88525	validation-rmse:0.41639	validation-SMAPE:66.50177
[100]	train-rmse:0.40794	train-SMAPE:65.37932	validation-rmse:0.41309	validation-SMAPE:66.01112
[110]	train-rmse:0.40452	train-SMAPE:64.96115	val

/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[10]	train-rmse:0.39402	train-SMAPE:64.98023	validation-rmse:0.40745	validation-SMAPE:66.49345
[20]	train-rmse:0.36523	train-SMAPE:60.63772	validation-rmse:0.38765	validation-SMAPE:63.04592
[30]	train-rmse:0.35149	train-SMAPE:58.79265	validation-rmse:0.37921	validation-SMAPE:61.78358
[40]	train-rmse:0.33794	train-SMAPE:56.96267	validation-rmse:0.37140	validation-SMAPE:60.62100
[50]	train-rmse:0.32706	train-SMAPE:55.55216	validation-rmse:0.36668	validation-SMAPE:59.92004
[60]	train-rmse:0.31941	train-SMAPE:54.53076	validation-rmse:0.36361	validation-SMAPE:59.44622
[70]	train-rmse:0.31204	train-SMAPE:53.61115	validation-rmse:0.36078	validation-SMAPE:59.02780
[80]	train-rmse:0.30572	train-SMAPE:52.81573	validation-rmse:0.35878	validation-SMAPE:58.75816
[90]	train-rmse:0.29896	train-SMAPE:51.90017	validation-rmse:0.35692	validation-SMAPE:58.46767
[100]	train-rmse:0.29300	train-SMAPE:51.13127	validation-rmse:0.35566	validation-SMAPE:58.30538
[110]	train-rmse:0.28719	train-SMAPE:50.38279	val

[550]	train-rmse:0.27152	train-SMAPE:48.39149	validation-rmse:0.34192	validation-SMAPE:56.37655
[560]	train-rmse:0.27020	train-SMAPE:48.21981	validation-rmse:0.34162	validation-SMAPE:56.32777
[570]	train-rmse:0.26920	train-SMAPE:48.09056	validation-rmse:0.34147	validation-SMAPE:56.30185
[580]	train-rmse:0.26832	train-SMAPE:47.97749	validation-rmse:0.34130	validation-SMAPE:56.27961
[590]	train-rmse:0.26745	train-SMAPE:47.86223	validation-rmse:0.34113	validation-SMAPE:56.25702
[600]	train-rmse:0.26650	train-SMAPE:47.74141	validation-rmse:0.34097	validation-SMAPE:56.22574
[610]	train-rmse:0.26553	train-SMAPE:47.61480	validation-rmse:0.34082	validation-SMAPE:56.20687
[620]	train-rmse:0.26442	train-SMAPE:47.46423	validation-rmse:0.34066	validation-SMAPE:56.19347
[630]	train-rmse:0.26327	train-SMAPE:47.31257	validation-rmse:0.34044	validation-SMAPE:56.16540
[640]	train-rmse:0.26224	train-SMAPE:47.18121	validation-rmse:0.34027	validation-SMAPE:56.14108
[650]	train-rmse:0.26141	train-SMAPE:47.

In [33]:
y_pred

array([-0.29749766,  0.55276406, -1.0714456 , ...,  0.73785084,
        0.9233028 , -0.27979472], dtype=float32)

In [34]:
# 反标准化 y_test
y_test_original = scaler_y.inverse_transform(y_test.values.reshape(-1, 1))

# 反标准化预测结果 y_pred
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

In [35]:
y_test_original

array([[ 92.],
       [158.],
       [ 60.],
       ...,
       [178.],
       [170.],
       [ 96.]])

In [36]:
y_pred_original

array([[106.68717 ],
       [151.08849 ],
       [ 66.271034],
       ...,
       [160.75386 ],
       [170.4383  ],
       [107.61163 ]], dtype=float32)

In [37]:
# 正确的写法
test_results = list(zip(y_test_original[:100], y_pred_original[:100]))  # 真实值和预测值

print("\n20条测试结果（真实值 vs 预测值）:")
for i, (true_value, pred_value) in enumerate(test_results[:40]):
    # 如果是多维数组，使用 .item() 转换为标量
    true_value = true_value.item() if isinstance(true_value, np.ndarray) else true_value
    pred_value = pred_value.item() if isinstance(pred_value, np.ndarray) else pred_value
    print(f"第{i+1}条: 真实值={true_value}, 预测值={pred_value:.2f}")



20条测试结果（真实值 vs 预测值）:
第1条: 真实值=92.0, 预测值=106.69
第2条: 真实值=158.0, 预测值=151.09
第3条: 真实值=60.0, 预测值=66.27
第4条: 真实值=216.0, 预测值=213.35
第5条: 真实值=158.0, 预测值=153.16
第6条: 真实值=174.0, 预测值=175.75
第7条: 真实值=183.0, 预测值=172.47
第8条: 真实值=61.0, 预测值=86.58
第9条: 真实值=89.0, 预测值=122.10
第10条: 真实值=161.0, 预测值=151.37
第11条: 真实值=170.0, 预测值=169.54
第12条: 真实值=166.0, 预测值=156.96
第13条: 真实值=131.0, 预测值=109.65
第14条: 真实值=119.0, 预测值=118.96
第15条: 真实值=85.0, 预测值=85.94
第16条: 真实值=134.0, 预测值=108.04
第17条: 真实值=172.0, 预测值=141.56
第18条: 真实值=75.0, 预测值=63.34
第19条: 真实值=73.0, 预测值=86.06
第20条: 真实值=26.0, 预测值=37.47
第21条: 真实值=37.0, 预测值=41.05
第22条: 真实值=129.0, 预测值=130.10
第23条: 真实值=54.0, 预测值=35.48
第24条: 真实值=129.0, 预测值=134.31
第25条: 真实值=70.0, 预测值=71.68
第26条: 真实值=169.0, 预测值=173.10
第27条: 真实值=101.0, 预测值=82.02
第28条: 真实值=31.0, 预测值=53.60
第29条: 真实值=74.0, 预测值=67.69
第30条: 真实值=91.0, 预测值=114.38
第31条: 真实值=140.0, 预测值=147.76
第32条: 真实值=70.0, 预测值=65.60
第33条: 真实值=74.0, 预测值=73.89
第34条: 真实值=83.0, 预测值=99.49
第35条: 真实值=125.0, 预测值=115.63
第36条: 真实值=156.0, 预测值=149.20
第37条: 真实值=1

In [38]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true)
    
    # 避免除以零，将分母中为零的项替换为一个小值
    denominator = np.where(denominator == 0, 1e-8, denominator)
    
    smape = 100 * np.mean(diff / denominator)
    return smape


def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    
    # 避免除以零，将 y_true 中的零值替换为一个小值
    y_true = np.where(y_true == 0, 1e-8, y_true)
    
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


# 假设 y_test 和 y_pred 已经是标准化反归一化后的数据
# 将其转换为一维数组以确保形状一致
y_test = np.array(y_test_original).ravel()
y_pred = np.array(y_pred_original).ravel()

# 评估指标
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = calculate_mape(y_test, y_pred)
smape = calculate_smape(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 打印结果
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')
print(f'R-squared (R²): {r2:.4f}')

Mean Squared Error (MSE): 311.5183
Root Mean Squared Error (RMSE): 17.6499
Mean Absolute Error (MAE): 13.1986
Mean Absolute Percentage Error (MAPE): 15.0148%
Symmetric Mean Absolute Percentage Error (SMAPE): 13.6369%
R-squared (R²): 0.8858


似乎对于较小值预测存在误差

## 保存模型

In [39]:
model.save_model("data_hh/result/model/归一化_xgboost_model_1000.json")
print("模型已保存为 xgboost_model_1000.json")

模型已保存为 xgboost_model_1000.json


## 超参数设置

## 不同特征重要程度测试

In [40]:
import xgboost as xgb
import matplotlib.pyplot as plt

# 假设 model 是训练好的 XGBoost 模型
xgb.plot_importance(model, importance_type='weight', title="Feature Importance (Weight)", height=0.5)
plt.show()

xgb.plot_importance(model, importance_type='gain', title="Feature Importance (Gain)", height=0.5)
plt.show()

xgb.plot_importance(model, importance_type='cover', title="Feature Importance (Cover)", height=0.5)
plt.show()

TypeError: can only be called with ndarray object

Error in callback <function _draw_all_if_interactive at 0x7fd4b6f31990> (for post_execute), with arguments args (),kwargs {}:


ValueError: object __array__ method not producing an array

ImportError: cannot import name 'ERR_IGNORE' from 'numpy.core.umath' (/data/zy/anaconda3/envs/machine/lib/python3.10/site-packages/numpy/core/umath.py)

<Figure size 640x480 with 1 Axes>